In [8]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import tensorflow.keras.layers as layers
from gan import GenLSTMd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
noise_dim = 2
input_size = 3
seq_len = 5
hidden_size = 7
batch_size = 11
n_lstm_layers = 1

In [13]:
lstm = layers.LSTM(input_shape=(seq_len, input_size), units=hidden_size, return_sequences=True, return_state=True)
input = tf.random.normal((batch_size, seq_len, input_size))
output, hidden_state, cell_state = lstm(input)
output.shape, hidden_state.shape, cell_state.shape

(TensorShape([11, 5, 7]), TensorShape([11, 7]), TensorShape([11, 7]))

In [14]:
output[:, -1, :] == hidden_state

<tf.Tensor: shape=(11, 7), dtype=bool, numpy=
array([[ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True]])>

In [10]:
# use hidden state and cell state as initial state
input = tf.random.normal((batch_size, seq_len, input_size))
output, hidden_state, cell_state = lstm(input, initial_state=[hidden_state, cell_state])
output.shape, hidden_state.shape, cell_state.shape

(TensorShape([11, 5, 7]), TensorShape([11, 7]), TensorShape([11, 7]))

In [40]:
noise = tf.random.normal((batch_size, seq_len-1, 1))
hist_x = tf.random.normal((batch_size, seq_len, input_size-2))
t = tf.range(seq_len, dtype=tf.float32)
t = tf.reshape(t, (1, seq_len, 1))
t = tf.tile(t, (batch_size, 1, 1))
t.shape

TensorShape([11, 5, 1])

In [41]:
batch_size = noise.shape[0] # noise shape: batch_size, seq_len, noise_dim
h = tf.zeros([batch_size, hidden_size])
c = tf.zeros([batch_size, hidden_size])
seq = tf.zeros([batch_size, 1, 1])

dts = t[:, 1:, :] - t[:, :-1, :]
if hist_x is not None: # feed in the historical data to get the hidden state
    diff_x = hist_x[:, 1:, :] - hist_x[:, :-1, :]
    input = tf.concat([diff_x, noise[:, :tf.shape(diff_x)[1], :], dts[:, :tf.shape(diff_x)[1], :]], axis=-1)
    output, h, c = lstm(input, initial_state=[h, c])
    noise = noise[:,tf.shape(diff_x)[1]:,:] # set the noise to start from the end of the historical data
    dts = dts[:,tf.shape(diff_x)[1]:,:] # continue from the last dt
    print(seq.shape, diff_x.shape)
    seq = tf.concat([seq, diff_x], axis=1)
    print(seq.shape)
else:
    output = tf.zeros([batch_size, 1, hidden_size])
len(seq), output.shape, noise.shape, dts.shape

(11, 1, 1) (11, 4, 1)
(11, 5, 1)


(11, TensorShape([11, 4, 7]), TensorShape([11, 0, 1]), TensorShape([11, 0, 1]))

In [11]:
noise_dim = 2
seq_dim = 1
seq_len = 13
hist_len = 5
hidden_size = 7
batch_size = 11

In [12]:
x = tf.random.normal((batch_size, seq_len, seq_dim))
noise = tf.random.normal((batch_size, seq_len-1, noise_dim))
t = tf.range(seq_len, dtype=tf.float32)
t = tf.reshape(t, (1, seq_len, 1))
t = tf.tile(t, (batch_size, 1, 1))
x = tf.concat([t, x], axis=-1)
model = GenLSTMd(noise_dim, seq_dim, seq_len, hist_len, hidden_size=hidden_size)
output = model(noise, x)
output.shape

Generated single step output shape: (11, 1, 1)
Generated single step output shape: (11, 1, 1)
Generated single step output shape: (11, 1, 1)
Generated single step output shape: (11, 1, 1)
Generated single step output shape: (11, 1, 1)
Generated single step output shape: (11, 1, 1)
Generated single step output shape: (11, 1, 1)
Generated single step output shape: (11, 1, 1)
Historical sequence shape: (11, 5, 1)
Generated sequence shape: (11, 8, 1)


TensorShape([11, 13, 2])

In [7]:
output[1]

<tf.Tensor: shape=(13, 2), dtype=float32, numpy=
array([[ 0.        ,  0.        ],
       [ 1.        ,  1.8988743 ],
       [ 2.        ,  0.9065581 ],
       [ 3.        ,  2.9653125 ],
       [ 4.        ,  2.6369126 ],
       [ 5.        ,  2.4417803 ],
       [ 6.        ,  2.2293181 ],
       [ 7.        ,  1.9813786 ],
       [ 8.        ,  1.6726481 ],
       [ 9.        ,  1.3360975 ],
       [10.        ,  0.9697833 ],
       [11.        ,  0.60960984],
       [12.        ,  0.23554277]], dtype=float32)>